# Lua III
## Metatables (Objects, sort of) & Review

## Metatables 
- Metatables are one of the newest additions to Lua
 - Added in Lua 5.0
- Metatables themselves are ordinary tables
 - They gain their special status through how we attach them to tables
- Metatables can be used to represent user-defined types
- All data types have metatables
 - Only the metatables of tables can be set and modified


## Basic Metatable Use


In [1]:
Window = {}
Window.prototype = {x = 0, y = 0, width = 100, height = 100}
Window.mt = {}

function Window.new(o)
    setmetatable(o,Window.mt)
    return o
end

Window.mt.__index = Window.prototype

w = Window.new({x = 10, y= 20})
z = Window.new{height = 100, width = 50}

In [2]:
w.width

100

In [3]:
z.x

0

## What can we do with Metatables
- Set Default values
- Limited Operator Overloading
 - Can only overload a small set of built in functions
 - No concept of method signature
- Reuse a metatable for multiple tables, ie a class
- Fake inheritance

## Setting Default Values
- To set default values when querying, we use the __\_\_index__ metamethod
 - What this actually does is provide a fall back table whose values are accses when the main table returns now for a given key
- As tables can hold functions, this allows a way to do inheritance

## Default Values Example

In [1]:
Building = {lat = 39.2555, long = 76.7113}
function Building.print(self)
    print("I'm located at " .. self.lat .. ", " .. self.long)
end
Building.__index = Building
UMBC = {}
setmetatable(UMBC,Building)

UMBC.print(UMBC)

I'm located at 39.2555, 76.7113

## Inheritance Example

In [2]:
House = {numberOfBathrooms = 35}
function House.printBaths(self)
    print("I have " .. self.numberOfBathrooms .. " bathrooms.")
end

setmetatable(House,Building)
T = {}
T.__index = House
whiteHouse = {lat = 38.8977, long = 77.0336}
setmetatable(whiteHouse,T)--{__index = House})

whiteHouse.print(whiteHouse)
whiteHouse.printBaths(whiteHouse)

I'm located at 38.8977, 77.0336
I have 35 bathrooms.

## Controlling the Creation of New Table Elements
- Just like the **__index** metamethod is called when an unknown element is accessed, the **__newindex** metamethod is used to update tables
- Defining this will prevent default behavior from occur
    - Causes tables to behave in a read-only like fashion
    - Can be bypassed using the `rawset` function

In [3]:
T.__newindex = function(t,k,v)
    print("No Updates Allowed")
end
whiteHouse.numberOfBathrooms = 4
whiteHouse.address = "1600 Pennsylvania Avenue"

No Updates Allowed
No Updates Allowed

In [4]:
T.__newindex = function(t,k,v)
    rawset(t,k,v)
end
whiteHouse.numberOfBathrooms = 4
whiteHouse.address = "1600 Pennsylvania Avenue"
whiteHouse:printBaths()

I have 4 bathrooms.

## Metamethod for Libraries
- A metaattribute or metamethod does not have to be limited to what Lua expects
- Several libraries can take advantage of metamethods to change their behavior.
    - **__tostring**
    - **__metatable**

In [5]:
print(whiteHouse)
T.__tostring = function(var)
    return "I'm located at " .. var.lat .. ", " .. var.long
end
print(whiteHouse)

table: 0x240bb70
I'm located at 38.8977, 77.0336

In [11]:
-- The __metatable Method is used to control the behavoir of the 
-- setmetatable and getmetatable functions
print(getmetatable(whiteHouse))
T.__metatable = "T"
print(getmetatable(whiteHouse))
setmetatable(whiteHouse,T)

[string "-- The __metatable Method is used to contro..."]:6: cannot change a protected metatable

## Operator Overloading
- To overload a given operator, set the corresponding _metamethod_
 - To overload +, define \_\_add (two underscores)
 - Each metatable only has one metamethod of each name
  - To account for differing operand types, a long sequence of __if__ statements may be needed

## Operator Overloading Example
- Suppose we have a metatable for the class of complex numbers

In [ ]:
--Based off Fabio Mascarenhas' example
mt = {}
function mt.__add(c1,c2)
    if(getmetatable(c1) ~= mt) then
        res = {r = c1 + c2.r , i = c2.i}
        setmetatable(res,mt)
        return res
    end
    if(getmetatable(c2) ~= mt) then
        res = {r = c1.r + c2, i = c1.i}
        setmetatable(res,mt)
        return res
    end
    res = {r = c1.r + c2.r , i = c1.i + c2.i}
    setmetatable(res,mt)
    return res
end

complex = {r = 1, i = 2}
complex2 = {r = 2, i = 4}
setmetatable(complex,mt)
setmetatable(complex2,mt)
print(5 + complex) 
print(5 - complex)

## Other Overloadable Operations
- **__mul**
- **__sub**
- **__div**
- **__unm**
- **__pow**
- **__eq**
- **__lt**
- **__le** 

## OOP in Lua
- We will discuss object oriented programming in detail over the next few classes
- In Lua, we can approximate an object by
 - Using metatables to define a class
 - Use the __:__ operator as a shorthand for passing the current value to a function
 

In [12]:
Animal = {species = 'Dog', noise = "woof"}

function Animal.makeNoise(animal)
    print(animal.species .. " goes " .. animal.noise)
end

Animal.makeNoise(Animal)


Dog goes woof

In [14]:
Animal2 = {species = "Dog" , noise = "woof"}

function Animal2:makeNoise(punct)
    print( self.species .. " goes " .. self.noise .. punct )
end

Animal2:makeNoise("!")
Animal2.makeNoise(Animal2,"!")

Dog goes woof!
Dog goes woof!

In [19]:
Animal3 = {species = "Dog" , noise = "woof"}

function Animal3.makeNoise(self)
    print( self.species .. " goes " .. self.noise )
end

Animal3:makeNoise()

Dog goes woof

## Object Constructors
- In Lua, the constructor becomes a function that
    - creates a table
    - sets the appropriate metatable
    - and returns the object

In [20]:
function Animal:new( m_species, m_noise)
    o = {species = m_species, noise = m_noise}
    setmetatable(o, self)
    self.__index = self
    return o
end

function Animal:makeNoise()
    print( self.species .. " goes " .. self.noise )
end

In [24]:
dog = Animal:new("Dog","Woof")
dog:makeNoise()

cat = Animal:new("Cat","Meow")
cat:makeNoise()

Dog goes Woof
Cat goes Meow

## Issues with OOP in Lua
- Can be cumbersome
- An object is still just a table
- Metatables can be changed
- Consider the output of the following:

In [16]:
type(Animal3)

table

In [17]:
Window = {}
Window.prototype = {x = 0, y = 0, width = 100, height = 100}
Window.mt = {}
Window.mt2 = {}

function Window.new(o)
    setmetatable(o,Window.mt)
    return o
end

Window.mt.__index = Window.prototype
Window.mt2.__index = {color = "red", shape = "circle"}

In [18]:
x = Window.new{height = 200}
print(x.x,x.y,x.width,x.height)
setmetatable(x,Window.mt2)
print(x.x,x.y,x.width,x.height)

x = {10, 20, 30, 40}
print(x[1])

0	0	100	200
nil	nil	nil	200
10

## Lua Wrap Up and Analysis
- To finish up our discussion of Lua, we are going to run through some things from chapters 5-10 of the textbook

## Names, Binding, and Scope
- Names
 - Case sensitivity?
   - Yes
 - Can reserved words be used?
   - No
 - What characters are allowed?
    - Starts with letters, can have underscores and numbers
- Binding 
 - When does type binding occur?
    - At assignment
- Scope
 - What is the default scope of a variable? 
   - Global 

## Data Types
- What are the data types?
 - Number, Boolean, String, Table, Nil, Function, Threads and User Data
- Can the user define their own data types?
  -  Yes, its cumbersome
- Arrays 
 - Are subscripts checked?
   - No
 - Can we use slicing?
   - No
 - How well supported are multidimensional arrays?
   - Not Very Well
- Are pointers accessible?
  - No
- Is there type checking?
 - No

## Expressions and Assignments
- Does Lua have operator precedence?
 - Yes
- Does Lua allow operator overloading?
 - To an Extent
- How are type conversions done?
 - Implicitly
- Does Lua have compound assignment operators (ie +=) ?
 - No

## Control Structures
- What is the syntax of an if statement?
 - if <> then <> end
- Does Lua have a multiple-select structure (ie switch)?
 - No
- What are Lua's counter controlled loops
 - for
- What are Lua's logic controlled loops?
 - while, repeat

## Subprograms
- What is the function syntax in Lua
 - function name(<>) <> end
- Does Lua allow functions to be passed as parameters of other functions?
 - Yes
- Are functions in Lua type-checked
 - No
- Can functions in Lua return more than one value?
 - Yes
- Can function definitions be nested?
 - Yes
- Does Lua allow closures?
 - Yes

## Lua Example Application
- A common task in natural language processing is to be able to read a file and calculate various statistics on the words in that file
- As an in-class exercise, we will write together a program that does the following
 - Reads in a text file and breaks it into words, based on spacing.
 - Counts the frequency of each word
 - Prints a the most common words in the file


## Lua Example Application 
### Step 1: Read in the file 

In [ ]:
words = {}
f = io.open("words.txt")
for line in f:lines() do
    for word in string.gmatch(line,"%w+") do
        table.insert(words,string.lower(word))
        --print(word)
    end
end

## Lua Example Application 
### Step 2: Count the words

In [ ]:
counts = {}
for _,word in pairs(words) do
    counts[word] = 0
    --print(word)
end

for _,word in pairs(words) do
    counts[word] = counts[word] + 1
    --print(word)
end

## Lua Example Application 
### Step 3: Print Most Common

In [ ]:
together = {}
for i,w in pairs(counts) do
    table.insert(together,{word = i, count = w})
end

table.sort(together,function(a,b) return a.count > b.count end)
for i,w in ipairs(together) do 
    print(w.word, w.count)
end
